# 交易策略回測 - 快速開始範例

這個筆記本示範如何使用我們的交易策略回測框架。

In [ ]:
# 導入必要的套件
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import fetch_data, plot_price_and_signals, print_strategy_summary
from strategies.sma_crossover import SMAcrossoverStrategy
from backtest_engine import BacktestEngine
from config.config import DEFAULT_SYMBOLS, get_config

## 1. 獲取市場資料

In [ ]:
# 選擇要測試的股票
symbol = 'AAPL'
period = '2y'  # 2年的資料

# 獲取資料
print(f"正在獲取 {symbol} 的資料...")
data = fetch_data(symbol, period=period)
print(f"獲取了 {len(data)} 筆資料")
data.head()

## 2. 創建交易策略

In [ ]:
# 創建簡單移動平均交叉策略
strategy = SMAcrossoverStrategy(
    name="SMA 20/50 交叉策略",
    short_window=20,  # 短期移動平均
    long_window=50,   # 長期移動平均
    initial_capital=100000  # 初始資金 10萬
)

print(f"策略名稱: {strategy.name}")
print(f"短期移動平均: {strategy.short_window} 天")
print(f"長期移動平均: {strategy.long_window} 天")
print(f"初始資金: ${strategy.initial_capital:,.2f}")

## 3. 運行回測

In [ ]:
# 創建回測引擎
backtest_config = get_config('backtest')
engine = BacktestEngine(
    initial_capital=backtest_config['initial_capital'],
    commission=backtest_config['commission']
)

# 運行回測
print("開始回測...")
result = engine.run_backtest(strategy, data, symbol)
print("回測完成!")

## 4. 分析結果

In [ ]:
# 顯示策略摘要
performance = result['performance']
print_strategy_summary(strategy.name, performance)

In [ ]:
# 顯示交易記錄
trades_df = pd.DataFrame(result['trades'])
if not trades_df.empty:
    print(f"\n總交易次數: {len(trades_df)}")
    print("\n前10筆交易:")
    print(trades_df.head(10))
else:
    print("\n沒有執行任何交易")

## 5. 視覺化分析

In [ ]:
# 繪製價格和交易信號
signals_data = result['signals_data']
plot_price_and_signals(signals_data, f"{strategy.name} - {symbol}")

In [ ]:
# 繪製投資組合價值變化
portfolio_value = result['portfolio_value']
if portfolio_value:
    plt.figure(figsize=(12, 6))
    
    # 投資組合價值
    portfolio_series = pd.Series(portfolio_value, index=data.index[:len(portfolio_value)])
    plt.plot(portfolio_series.index, portfolio_series.values, label='投資組合價值', linewidth=2)
    
    # 買入持有策略比較
    buy_hold_value = (data['Close'] / data['Close'].iloc[0]) * strategy.initial_capital
    plt.plot(data.index, buy_hold_value, label='買入持有', linewidth=2, alpha=0.7)
    
    plt.title(f'{strategy.name} - 投資組合價值 vs 買入持有')
    plt.xlabel('日期')
    plt.ylabel('價值 ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 計算買入持有的績效
    buy_hold_return = (buy_hold_value.iloc[-1] / strategy.initial_capital) - 1
    strategy_return = performance.get('total_return', 0)
    
    print(f"\n績效比較:")
    print(f"策略總回報: {strategy_return:.2%}")
    print(f"買入持有回報: {buy_hold_return:.2%}")
    print(f"超額回報: {(strategy_return - buy_hold_return):.2%}")

## 6. 測試多個股票

In [ ]:
# 測試多個熱門股票
test_symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'SPY']

print("正在測試多個股票...")
multi_results = {}

for test_symbol in test_symbols:
    print(f"\n測試 {test_symbol}...")
    
    # 獲取資料
    test_data = fetch_data(test_symbol, period='1y')
    
    if not test_data.empty:
        # 創建新的策略實例
        test_strategy = SMAcrossoverStrategy(
            name=f"SMA 20/50 - {test_symbol}",
            short_window=20,
            long_window=50
        )
        
        # 運行回測
        test_result = engine.run_backtest(test_strategy, test_data, test_symbol)
        multi_results[test_symbol] = test_result['performance']
        
        # 顯示簡要結果
        perf = test_result['performance']
        total_return = perf.get('total_return', 0)
        sharpe = perf.get('sharpe_ratio', 0)
        print(f"  總回報: {total_return:.2%}, 夏普比率: {sharpe:.3f}")
    else:
        print(f"  無法獲取 {test_symbol} 的資料")

In [ ]:
# 顯示多股票回測摘要
if multi_results:
    summary_data = []
    for symbol, perf in multi_results.items():
        summary_data.append({
            '股票代碼': symbol,
            '總回報 (%)': f"{perf.get('total_return', 0):.2%}",
            '年化回報 (%)': f"{perf.get('annualized_return', 0):.2%}",
            '波動率 (%)': f"{perf.get('volatility', 0):.2%}",
            '夏普比率': f"{perf.get('sharpe_ratio', 0):.3f}",
            '最大回撤 (%)': f"{perf.get('max_drawdown', 0):.2%}",
            '交易次數': perf.get('total_trades', 0)
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\n多股票回測摘要:")
    print(summary_df.to_string(index=False))

## 總結

這個範例展示了如何:

1. **獲取市場資料** - 使用 Yahoo Finance API
2. **創建交易策略** - 實現簡單移動平均交叉策略
3. **執行回測** - 使用自建的回測引擎
4. **分析結果** - 計算各種績效指標
5. **視覺化** - 繪製價格、信號和投資組合價值
6. **比較分析** - 測試多個股票的表現

### 下一步可以做什麼:

- 實現更多交易策略（RSI、MACD、布林通道等）
- 加入風險管理（停損、停利）
- 優化策略參數
- 加入更多績效指標
- 實現組合投資策略
- 加入交易成本和滑價分析